# Beginner Challenge

After getting accustomed to using TKET, we will use a sample circuit given [here](https://github.com/spendierk/ethz-hackathon22/tree/main/benchmarking/circuits) to test some optimization features and run it on some different backends provided by IBM Quantum.

In [30]:
#Pytket imports
from pytket import Circuit, Qubit
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import AerBackend
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket.qasm import circuit_to_qasm, circuit_from_qasm
from pytket.circuit import OpType

#Optimization imports
from pytket.passes import *
from pytket.predicates import CompilationUnit

#Other package imports for QC
from qiskit import QuantumCircuit

#For graphing and basic math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi

#Sanity check
backend = AerBackend()
backend.required_predicates

[NoSymbolsPredicate,
 GateSetPredicate:{ SX Y ZZPhase RangePredicate X YYPhase Z XXPhase S Sdg T Tdg SXdg CnX Barrier CCX H Rx Unitary1qBox Ry Unitary2qBox Rz U3 U2 U1 TK1 CX CY CZ CU1 CU3 SWAP CSWAP noop Measure Reset PhasedX }]

In [6]:
#Import qasm file
qasmfile = "8d1eb48e-d667-11ea-9cd7-38f9d36dfbf2.qasm"
qc = circuit_from_qasm(qasmfile) #generates circuit

In [7]:
#Draw with TKET
render_circuit_jupyter(qc)

In [8]:
#Draw in Qiskit
qc_qiskit = tk_to_qiskit(qc)
qc_qiskit.draw()

┌───────────┐  ┌────────────┐┌────────────┐┌─────────┐┌─────────┐»
q_0: ┤ Rz(6.263) ├──┤ Rx(2.5934) ├┤ Rz(2.9639) ├┤ Rz(π/2) ├┤ Rz(π/2) ├»
     ├───────────┴┐┌┴────────────┤├────────────┤└─────────┘└─────────┘»
q_1: ┤ Rz(2.1936) ├┤ Rx(0.48946) ├┤ Rz(2.5596) ├──────────────────────»
     └────────────┘└─────────────┘└────────────┘                      »
c: 2/═════════════════════════════════════════════════════════════════»
                                                                      »
«     ┌─────────┐┌─────────┐   ┌─────────┐  ┌─────────┐  ┌─────────┐»
«q_0: ┤ Rx(π/2) ├┤ Rz(π/2) ├─■─┤ Rz(π/2) ├──┤ Rx(π/2) ├──┤ Rz(π/2) ├»
«     └─────────┘└─────────┘ │ ├─────────┴┐┌┴─────────┴─┐├─────────┤»
«q_1: ───────────────────────■─┤ Rz(3π/2) ├┤ Rx(5.4177) ├┤ Rz(π/2) ├»
«                              └──────────┘└────────────┘└─────────┘»
«c: 2/══════════════════════════════════════════════════════════════»
«                                                                   »
«     ┌─────────────┐                         ┌─────────┐┌─────────┐┌─────────┐»
«q_0: ┤ Rz(0.22002) ├───────────────────────■─┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(π/2) ├»
«     └─┬─────────┬─┘┌─────────┐┌─────────┐ │ ├─────────┤├─────────┤├─────────┤»
«q_1: ──┤ Rz(π/2) ├──┤ Rx(π/2) ├┤ Rz(π/2) ├─■─┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(π/2) ├»
«       └─────────┘  └─────────┘└─────────┘   └─────────┘└─────────┘└─────────┘»
«c: 2/═════════════════════════════════════════════════════════════════════════»
«                                                                              »
«                                             ┌─────────┐  ┌─────────┐  »
«q_0: ──────────────────────────────────────■─┤ Rz(π/2) ├──┤ Rx(π/2) ├──»
«     ┌──────────┐┌────────────┐┌─────────┐ │ ├─────────┴┐┌┴─────────┴─┐»
«q_1: ┤ Rz(3π/2) ├┤ Rx(1.5622) ├┤ Rz(π/2) ├─■─┤ Rz(3π/2) ├┤ Rz(3.1643) ├»
«     └──────────┘└────────────┘└─────────┘   └──────────┘└────────────┘»
«c: 2/══════════════════════════════════════════════════════════════════»
«                                                                       »
«      ┌─────────┐  ┌────────────┐ ┌────────────┐┌────────────┐┌────────────┐»
«q_0: ─┤ Rz(π/2) ├──┤ Rz(4.1536) ├─┤ Rx(1.1211) ├┤ Rz(2.2397) ├┤ Rz(1.6988) ├»
«     ┌┴─────────┴─┐├────────────┤┌┴────────────┤├────────────┤├────────────┤»
«q_1: ┤ Rx(2.9469) ├┤ Rz(4.0259) ├┤ Rz(0.87401) ├┤ Rx(1.9507) ├┤ Rz(2.5005) ├»
«     └────────────┘└────────────┘└─────────────┘└────────────┘└────────────┘»
«c: 2/═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«     ┌────────────┐┌────────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
«q_0: ┤ Rx(1.2824) ├┤ Rz(3.8503) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(π/2) ├»
«     └────────────┘└────────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«q_1: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«        ┌─────────┐  ┌─────────┐  ┌─────────┐┌─────────────┐           »
«q_0: ─■─┤ Rz(π/2) ├──┤ Rx(π/2) ├──┤ Rz(π/2) ├┤ Rz(0.12851) ├───────────»
«      │ ├─────────┴┐┌┴─────────┴─┐├─────────┤└─┬─────────┬─┘┌─────────┐»
«q_1: ─■─┤ Rz(3π/2) ├┤ Rx(5.6561) ├┤ Rz(π/2) ├──┤ Rz(π/2) ├──┤ Rx(π/2) ├»
«        └──────────┘└────────────┘└─────────┘  └─────────┘  └─────────┘»
«c: 2/══════════════════════════════════════════════════════════════════»
«                                                                       »
«                   ┌─────────┐┌─────────┐┌─────────┐                          »
«q_0: ────────────■─┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(π/2) ├──────────────────────────»
«     ┌─────────┐ │ ├─────────┤├─────────┤├─────────┤┌──────────┐┌────────────┐»
«q_1: ┤ Rz(π/2) ├─■─┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(π/2) 

In [28]:
#Examine circuit depth, since noise will increase with more high-depth circuits
print("Qiskit circuit depth reading: " + str(qc_qiskit.depth()))
print("TKET circuit depth reading: " + str(qc.depth())) 

Qiskit circuit depth reading: 56
TKET circuit depth reading: 56


We've verified that both `pytket` and `qiskit` render the same, very long circuit. Now to try and optimize this:

In [18]:
seqpass = SequencePass([CommuteThroughMultis(), RemoveRedundancies()])
reppass = RepeatPass(seqpass)
cu = CompilationUnit(qc)
reppass.apply(cu)
circ = cu.circuit
render_circuit_jupyter(circ)

In [19]:
#Verifying same result in qiskit
circ_qiskit = tk_to_qiskit(circ)
circ_qiskit.draw()

┌───────────┐  ┌────────────┐┌────────────┐┌─────────┐┌───────┐   »
q_0: ┤ Rz(6.263) ├──┤ Rx(2.5934) ├┤ Rz(6.1055) ├┤ Rx(π/2) ├┤ Rz(π) ├─■─»
     ├───────────┴┐┌┴────────────┤├───────────┬┘└─────────┘└───────┘ │ »
q_1: ┤ Rz(2.1936) ├┤ Rx(0.48946) ├┤ Rz(7.272) ├──────────────────────■─»
     └────────────┘└─────────────┘└───────────┘                        »
c: 2/══════════════════════════════════════════════════════════════════»
                                                                       »
«      ┌─────────┐  ┌────────────┐                        ┌─────────┐ »
«q_0: ─┤ Rx(π/2) ├──┤ Rz(3.3616) ├─────────────────────■──┤ Rx(π/2) ├─»
«     ┌┴─────────┴─┐└─┬───────┬──┘┌─────────┐┌───────┐ │ ┌┴─────────┴┐»
«q_1: ┤ Rx(5.4177) ├──┤ Rz(π) ├───┤ Rx(π/2) ├┤ Rz(π) ├─■─┤ Rx(3.133) ├»
«     └────────────┘  └───────┘   └─────────┘└───────┘   └───────────┘»
«c: 2/════════════════════════════════════════════════════════════════»
«                                                                     »
«       ┌───────┐       ┌─────────┐  ┌────────────┐┌────────────┐┌────────────┐»
«q_0: ──┤ Rz(π) ├────■──┤ Rx(π/2) ├──┤ Rz(5.7244) ├┤ Rx(1.1211) ├┤ Rz(3.9384) ├»
«     ┌─┴───────┴──┐ │ ┌┴─────────┴─┐├────────────┤├────────────┤├────────────┤»
«q_1: ┤ Rz(3.1643) ├─■─┤ Rx(2.9469) ├┤ Rz(4.8999) ├┤ Rx(1.9507) ├┤ Rz(7.2129) ├»
«     └────────────┘   └────────────┘└────────────┘└────────────┘└────────────┘»
«c: 2/═════════════════════════════════════════════════════════════════════════»
«                                                                              »
«     ┌────────────┐┌────────────┐┌─────────┐┌───────┐    ┌─────────┐  »
«q_0: ┤ Rx(1.2824) ├┤ Rz(6.9919) ├┤ Rx(π/2) ├┤ Rz(π) ├─■──┤ Rx(π/2) ├──»
«     └────────────┘└────────────┘└─────────┘└───────┘ │ ┌┴─────────┴─┐»
«q_1: ─────────────────────────────────────────────────■─┤ Rx(5.6561) ├»
«                                                        └────────────┘»
«c: 2/═════════════════════════════════════════════════════════════════»
«                                                                      »
«     ┌────────────┐                        ┌─────────┐    ┌───────┐      »
«q_0: ┤ Rz(3.2701) ├─────────────────────■──┤ Rx(π/2) ├────┤ Rz(π) ├────■─»
«     └─┬───────┬──┘┌─────────┐┌───────┐ │ ┌┴─────────┴─┐┌─┴───────┴──┐ │ »
«q_1: ──┤ Rz(π) ├───┤ Rx(π/2) ├┤ Rz(π) ├─■─┤ Rx(2.6321) ├┤ Rz(5.4061) ├─■─»
«       └───────┘   └─────────┘└───────┘   └────────────┘└────────────┘   »
«c: 2/════════════════════════════════════════════════════════════════════»
«                                                                         »
«      ┌─────────┐  ┌────────────┐┌────────────┐┌────────────┐
«q_0: ─┤ Rx(π/2) ├──┤ Rz(4.9181) ├┤ Rx(1.7392) ├┤ Rz(1.7114) ├
«     ┌┴─────────┴─┐├────────────┤└────────────┘└────────────┘
«q_1: ┤ Rx(1.4306) ├┤ Rz(3.0463) ├────────────────────────────
«     └────────────┘└────────────┘                            
«c: 2/════════════════════════════════════════════════════════
«

In [27]:
print("Qiskit circuit depth reading: " + str(circ_qiskit.depth()))
print("TKET circuit depth reading: " + str(circ.depth()))

Qiskit circuit depth reading: 35
TKET circuit depth reading: 35


Controlled-NOT (CX) gates are infamous generators of noise since they entangle the quantum states of two qubits. Therefore, in order to limit noise, we examine the optimization possibilities by eliminating CX gates.

In [45]:
def removeCX(circuit):
    return circ.n_gates_of_type(OpType.CX) == 0

custom_pass = RepeatUntilSatisfiedPass(seqpass, removeCX)
cu = CompilationUnit(circ)
custom_pass.apply(cu)
circ1 = cu.circuit
render_circuit_jupyter(circ1)

In [46]:
print("TKET circuit depth reading: " + str(circ1.depth()))

TKET circuit depth reading: 35


The circuit depth stayed the same, however we have eliminated CX gates so there's a high probability that each operation has a higher gate fidelity. 

### Running the Optimized Circuit on IBM Quantum Chips

When a circuit is sent to an IBM Quantum Chip, there's a lot of optimization that gets done automatically. One way we can test if there are any further ways to optimize circuits is to send the job to IBM and print out the transpiled circuit that got optimized for a NISQ processor. Not only will this help further down the line when we want to apply as many optimization tricks as possible to the large LiH circuit, it will also provide insight to how we can optimize a circuit in general for quantum chips that don't already have some optimization protocols automatically applied. 

In [43]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator


IBMQ.save_account('d291ba136a052f06bd2f6c31aaf17c15248c077043a37910069fb39ac0c50588ff154b384bf71eafcca340589474664168c111e62f4a3ac1b62331e7f5e880a6') # Save TOKEN to disk
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers
provider = IBMQ.load_account()

configrc.store_credentials:WARNING:2022-08-15 22:24:53,670: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-08-15 22:24:53,864: Credentials are already in use. The existing account in the session will be replaced.
ibmqfactory.load_account:WARNING:2022-08-15 22:24:56,889: Credentials are already in use. The existing account in the session will be replaced.


In [47]:
circ1_qiskit = tk_to_qiskit(circ1)
circ1_qiskit.draw()

┌───────────┐  ┌────────────┐┌────────────┐┌─────────┐┌───────┐   »
q_0: ┤ Rz(6.263) ├──┤ Rx(2.5934) ├┤ Rz(6.1055) ├┤ Rx(π/2) ├┤ Rz(π) ├─■─»
     ├───────────┴┐┌┴────────────┤├───────────┬┘└─────────┘└───────┘ │ »
q_1: ┤ Rz(2.1936) ├┤ Rx(0.48946) ├┤ Rz(7.272) ├──────────────────────■─»
     └────────────┘└─────────────┘└───────────┘                        »
c: 2/══════════════════════════════════════════════════════════════════»
                                                                       »
«      ┌─────────┐  ┌────────────┐                        ┌─────────┐ »
«q_0: ─┤ Rx(π/2) ├──┤ Rz(3.3616) ├─────────────────────■──┤ Rx(π/2) ├─»
«     ┌┴─────────┴─┐└─┬───────┬──┘┌─────────┐┌───────┐ │ ┌┴─────────┴┐»
«q_1: ┤ Rx(5.4177) ├──┤ Rz(π) ├───┤ Rx(π/2) ├┤ Rz(π) ├─■─┤ Rx(3.133) ├»
«     └────────────┘  └───────┘   └─────────┘└───────┘   └───────────┘»
«c: 2/════════════════════════════════════════════════════════════════»
«                                                                     »
«       ┌───────┐       ┌─────────┐  ┌────────────┐┌────────────┐┌────────────┐»
«q_0: ──┤ Rz(π) ├────■──┤ Rx(π/2) ├──┤ Rz(5.7244) ├┤ Rx(1.1211) ├┤ Rz(3.9384) ├»
«     ┌─┴───────┴──┐ │ ┌┴─────────┴─┐├────────────┤├────────────┤├────────────┤»
«q_1: ┤ Rz(3.1643) ├─■─┤ Rx(2.9469) ├┤ Rz(4.8999) ├┤ Rx(1.9507) ├┤ Rz(7.2129) ├»
«     └────────────┘   └────────────┘└────────────┘└────────────┘└────────────┘»
«c: 2/═════════════════════════════════════════════════════════════════════════»
«                                                                              »
«     ┌────────────┐┌────────────┐┌─────────┐┌───────┐    ┌─────────┐  »
«q_0: ┤ Rx(1.2824) ├┤ Rz(6.9919) ├┤ Rx(π/2) ├┤ Rz(π) ├─■──┤ Rx(π/2) ├──»
«     └────────────┘└────────────┘└─────────┘└───────┘ │ ┌┴─────────┴─┐»
«q_1: ─────────────────────────────────────────────────■─┤ Rx(5.6561) ├»
«                                                        └────────────┘»
«c: 2/═════════════════════════════════════════════════════════════════»
«                                                                      »
«     ┌────────────┐                        ┌─────────┐    ┌───────┐      »
«q_0: ┤ Rz(3.2701) ├─────────────────────■──┤ Rx(π/2) ├────┤ Rz(π) ├────■─»
«     └─┬───────┬──┘┌─────────┐┌───────┐ │ ┌┴─────────┴─┐┌─┴───────┴──┐ │ »
«q_1: ──┤ Rz(π) ├───┤ Rx(π/2) ├┤ Rz(π) ├─■─┤ Rx(2.6321) ├┤ Rz(5.4061) ├─■─»
«       └───────┘   └─────────┘└───────┘   └────────────┘└────────────┘   »
«c: 2/════════════════════════════════════════════════════════════════════»
«                                                                         »
«      ┌─────────┐  ┌────────────┐┌────────────┐┌────────────┐
«q_0: ─┤ Rx(π/2) ├──┤ Rz(4.9181) ├┤ Rx(1.7392) ├┤ Rz(1.7114) ├
«     ┌┴─────────┴─┐├────────────┤└────────────┘└────────────┘
«q_1: ┤ Rx(1.4306) ├┤ Rz(3.0463) ├────────────────────────────
«     └────────────┘└────────────┘                            
«c: 2/════════════════════════════════════════════════════════
«

In [ ]:
real_device = provider.get_backend('ibmq_belem') #pick a device
job = execute(circ1_qiskit, real_device, shots=1024)
result = job.result() #execute circuit defined previously
belem_counts = result.get_counts()